In [ ]:
1. Código para traducción automática mediante Hugging Face

In [ ]:
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer

# 1. Cargar datos preprocesados
df = pd.read_csv('normalized_terms.csv')

# 2. Configurar modelo de traducción
model_name = "Helsinki-NLP/opus-mt-en-es"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# 3. Función de traducción con post-procesamiento
def translate_term(term: str) -> str:
    # Tokenizar y traducir
    inputs = tokenizer(term, return_tensors="pt", truncation=True)
    outputs = model.generate(**inputs)
    translated = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Post-procesamiento para términos médicos con problemas
    translated = translated.replace("aprobación", "acceso").replace("aislamiento", "exclusión")

    return translated

# 4. Aplicar a todo el dataset
df['Translated Term'] = df['Normalized Term'].apply(translate_term)

# 5. Guardar resultados
df.to_csv('translated_terms.csv', index=False)
print("¡Traducción completada! Resultados en 'translated_terms.csv'")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/826k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:177: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/312M [00:00<?, ?B/s]

¡Traducción completada! Resultados en 'translated_terms.csv'


2. CSV con normalizaciones, traducciones y códigos SDH

In [ ]:
import pandas as pd

# Cargar los archivos para inspeccionar las columnas
translated_df = pd.read_csv('translated_terms.csv')
codigos_df = pd.read_csv('todos_codigos_z55_z65_completo.csv')

translated_columns = translated_df.columns.tolist()
codigos_columns = codigos_df.columns.tolist()

translated_columns, codigos_columns


(['ID', 'Normalized Term', 'Translated Term'], ['Código', 'Descripción'])

In [ ]:
3. Mapeo ontológico

In [ ]:
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Cargar modelo de lenguaje de spaCy para español
nlp = spacy.load("es_core_news_md")

# Función de preprocesamiento: minúsculas, lematización, sin stopwords ni puntuación
def preprocess(text):
    doc = nlp(str(text).lower())
    tokens = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]
    return ' '.join(tokens)

# Preprocesar las columnas relevantes
translated_df['processed_term'] = translated_df['Translated Term'].apply(preprocess)
codigos_df['processed_desc'] = codigos_df['Descripción'].apply(preprocess)

# Vectorización TF-IDF
vectorizer = TfidfVectorizer().fit(codigos_df['processed_desc'])
desc_vectors = vectorizer.transform(codigos_df['processed_desc'])

# Mapeo basado en similitud de coseno
mapped_codes = []
for term in translated_df['processed_term']:
    term_vector = vectorizer.transform([term])
    similarities = cosine_similarity(term_vector, desc_vectors).flatten()
    best_match_idx = similarities.argmax()
    best_score = similarities[best_match_idx]
    if best_score > 0.5:
        codigo = codigos_df.iloc[best_match_idx]['Código']
        mapped_codes.append(codigo)
    else:
        mapped_codes.append("no-map")

# Añadir los resultados al DataFrame
translated_df['mapped_code'] = mapped_codes

# Guardar el resultado
output_path = 'mapeo_ontologico_resultado.csv'
translated_df.to_csv(output_path, index=False)

output_path



'mapeo_ontologico_resultado.csv'

4. Validacion

In [ ]:
!pip install -U spacy
!python -m spacy download es_core_news_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 80.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import pandas as pd
import spacy
import os
from pathlib import Path

# Cargar modelo en español
nlp = spacy.load("es_core_news_sm")

# Leer los términos traducidos automáticamente
translated_df = pd.read_csv("translated_terms.csv")
translated_terms = translated_df['Translated Term'].dropna().tolist()

# Leer los términos oficiales
official_df = pd.read_csv("todos_codigos_z55_z65_completo.csv")
official_terms = official_df['Descripción'].dropna().tolist()

# Función para lematizar listas de frases
def lemmatize_terms(term_list):
    lemmatized = []
    for term in term_list:
        doc = nlp(term.lower())
        lemmatized.append(" ".join([token.lemma_ for token in doc if not token.is_punct and not token.is_stop]))
    return lemmatized

lemmatized_translated = lemmatize_terms(translated_terms)
lemmatized_official = lemmatize_terms(official_terms)

# Para comprobar coincidencias
coincidencias = set(lemmatized_translated).intersection(set(lemmatized_official))
print(f"\n▶️ Coincidencias entre términos traducidos y oficiales: {len(coincidencias)}")
for match in coincidencias:
    print("-", match)

# Leer los 20 archivos .txt de una carpeta y buscar menciones
txt_folder = Path("./txts/")
txt_files = list(txt_folder.glob("*.txt"))

# Función para lematizar texto completo
def lemmatize_text(text):
    doc = nlp(text.lower())
    return " ".join([token.lemma_ for token in doc if not token.is_punct and not token.is_stop])

# Buscar términos traducidos y oficiales en cada archivo
resultados = []

for file in txt_files:
    with open(file, "r", encoding="utf-8") as f:
        content = f.read()
    lemmatized_content = lemmatize_text(content)

    encontrados = {
        "archivo": file.name,
        "traducidos_encontrados": [],
        "oficiales_encontrados": []
    }

    for term in lemmatized_translated:
        if term in lemmatized_content:
            encontrados["traducidos_encontrados"].append(term)

    for term in lemmatized_official:
        if term in lemmatized_content:
            encontrados["oficiales_encontrados"].append(term)

    resultados.append(encontrados)

# Mostrar resumen
for r in resultados:
    print(f"\n📄 Archivo: {r['archivo']}")
    print(f"🔸 Términos traducidos encontrados: {r['traducidos_encontrados']}")
    print(f"🔹 Términos oficiales encontrados: {r['oficiales_encontrados']}")



▶️ Coincidencias entre términos traducidos y oficiales: 2
- exposición ruido
- exposición radiación

📄 Archivo: 10.txt
🔸 Términos traducidos encontrados: ['desempleo', 'ingreso bajo', 'problema económico', 'aislamiento social']
🔹 Términos oficiales encontrados: []

📄 Archivo: 3.txt
🔸 Términos traducidos encontrados: []
🔹 Términos oficiales encontrados: []

📄 Archivo: 14.txt
🔸 Términos traducidos encontrados: ['desempleo', 'ingreso bajo']
🔹 Términos oficiales encontrados: []

📄 Archivo: 19.txt
🔸 Términos traducidos encontrados: []
🔹 Términos oficiales encontrados: []

📄 Archivo: 11.txt
🔸 Términos traducidos encontrados: []
🔹 Términos oficiales encontrados: []

📄 Archivo: 8.txt
🔸 Términos traducidos encontrados: []
🔹 Términos oficiales encontrados: []

📄 Archivo: 15.txt
🔸 Términos traducidos encontrados: []
🔹 Términos oficiales encontrados: []

📄 Archivo: 5.txt
🔸 Términos traducidos encontrados: ['desempleo', 'ingreso bajo', 'aislamiento social']
🔹 Términos oficiales encontrados: []

📄 

6. Coincidencias threshold >5 y otras

In [ ]:
import os
import glob
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 1. Leer el Excel con los términos traducidos
df = pd.read_excel("SDOH_Zcodes_traducciones.xlsx")
terminos = df["Español"].dropna().str.lower().tolist()

# 2. Leer todos los archivos .txt en la carpeta "TXTS"
txt_folder = "txts"
textos = []
archivos = []

for filepath in glob.glob(os.path.join(txt_folder, "*.txt")):
    with open(filepath, "r", encoding="utf-8") as f:
        contenido = f.read().lower()
        textos.append(contenido)
        archivos.append(os.path.basename(filepath))

# 3. Lista de stopwords personalizada (como lista)
stopwords_custom = [
    'de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'se', 'del', 'las',
    'por', 'un', 'para', 'con', 'no', 'una', 'su', 'al', 'lo', 'como',
    'más', 'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí', 'porque', 'esta',
    'entre', 'cuando', 'también', 'otros', 'fue', 'han', 'ser', 'hay',
    'todo', 'ha', 'está', 'están', 'desde', 'tiene', 'donde', 'mismo',
    'estos', 'ni', 'nos', 'durante', 'uno', 'les', 'ella', 'tanto'
]

# 4. Crear el corpus completo
corpus = textos + terminos
vectorizer = TfidfVectorizer(ngram_range=(1, 3), stop_words=stopwords_custom)
X = vectorizer.fit_transform(corpus)

# 5. Separar matrices
X_textos = X[:len(textos)]
X_terminos = X[len(textos):]

# 6. Calcular similitud coseno entre textos y términos
similitudes = cosine_similarity(X_textos, X_terminos)

# 7. Recoger coincidencias
umbral = 0.5
coincidencias = []

for i, archivo in enumerate(archivos):
    scores = list(enumerate(similitudes[i]))

    # Coincidencias por encima del umbral
    for j, score in scores:
        if score >= umbral:
            coincidencias.append({
                "archivo": archivo,
                "termino_diccionario": terminos[j],
                "similitud": round(score, 3),
                "tipo": "por_umbral"
            })

    # Las 3 mejores coincidencias, aunque estén por debajo del umbral
    top3 = sorted(scores, key=lambda x: x[1], reverse=True)[:3]
    for j, score in top3:
        coincidencias.append({
            "archivo": archivo,
            "termino_diccionario": terminos[j],
            "similitud": round(score, 3),
            "tipo": "top3"
        })

# 8. Crear DataFrame y eliminar duplicados exactos
df_resultado = pd.DataFrame(coincidencias).drop_duplicates()

# 9. Guardar en CSV
df_resultado.to_csv("coincidencias_sdoh_tfidf_top3.csv", index=False, encoding="utf-8")
print("✅ CSV generado: coincidencias_sdoh_tfidf_top3.csv")


✅ CSV generado: coincidencias_sdoh_tfidf_top3.csv


In [ ]:
7. Caso práctico concreto

In [ ]:
# Código mejorado con integración de sinónimos y lematización
import os
import glob
import pandas as pd
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Cargar modelo de spaCy para lematizar en español
nlp = spacy.load("es_core_news_sm")

# Diccionario de sinónimos lematizados para los 3 SDH clave
sdh_sinonimos = {
    "pobreza extrema": [
        "pobreza", "pobre", "extrema pobreza", "miseria", "vulnerabilidad económica", "carencia"
    ],
    "acceso limitado a alimentos o agua potable": [
        "malnutrición", "desnutrición", "hambre", "subalimentación", "carencia alimentaria",
        "agua no potable", "falta de agua", "falta de alimentos", "falta de acceso", "falta acceso alimentos"
    ],
    "desigualdad socioeconómica": [
        "desigualdad", "inequidad", "brecha económica", "brecha social", "exclusión social",
        "desigualdad de ingresos", "injusticia social"
    ]
}

# Aplanar sinónimos y añadir al listado de términos
sinonimos_extra = [t for sublist in sdh_sinonimos.values() for t in sublist]

# 1. Leer el Excel con los términos traducidos
df = pd.read_excel("SDOH_Zcodes_traducciones.xlsx")
terminos = df["Español"].dropna().str.lower().tolist()
terminos.extend(sinonimos_extra)
terminos = list(set(terminos))  # Eliminar duplicados

# Función de lematización
def lematizar(texto):
    return " ".join([token.lemma_ for token in nlp(texto) if not token.is_punct and not token.is_space])

# 2. Leer todos los archivos .txt en la carpeta "caso"
txt_folder = "caso"
textos = []
archivos = []

for filepath in glob.glob(os.path.join(txt_folder, "*.txt")):
    with open(filepath, "r", encoding="utf-8") as f:
        contenido = f.read().lower()
        contenido_lematizado = lematizar(contenido)
        textos.append(contenido_lematizado)
        archivos.append(os.path.basename(filepath))

# 3. Lista de stopwords personalizada
stopwords_custom = [
    'de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'se', 'del', 'las',
    'por', 'un', 'para', 'con', 'no', 'una', 'su', 'al', 'lo', 'como',
    'más', 'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí', 'porque', 'esta',
    'entre', 'cuando', 'también', 'otros', 'fue', 'han', 'ser', 'hay',
    'todo', 'ha', 'está', 'están', 'desde', 'tiene', 'donde', 'mismo',
    'estos', 'ni', 'nos', 'durante', 'uno', 'les', 'ella', 'tanto'
]

# 4. Lematizar también los términos del diccionario
terminos_lematizados = [lematizar(t) for t in terminos]

# 5. Crear el corpus completo
corpus = textos + terminos_lematizados
vectorizer = TfidfVectorizer(ngram_range=(1, 3), stop_words=stopwords_custom)
X = vectorizer.fit_transform(corpus)

# 6. Separar matrices
X_textos = X[:len(textos)]
X_terminos = X[len(textos):]

# 7. Calcular similitud coseno entre textos y términos
similitudes = cosine_similarity(X_textos, X_terminos)

# 8. Recoger coincidencias
umbral = 0.5
coincidencias = []

for i, archivo in enumerate(archivos):
    scores = list(enumerate(similitudes[i]))

    for j, score in scores:
        if score >= umbral:
            coincidencias.append({
                "archivo": archivo,
                "termino_diccionario": terminos[j],
                "similitud": round(score, 3),
                "tipo": "por_umbral"
            })

    top3 = sorted(scores, key=lambda x: x[1], reverse=True)[:3]
    for j, score in top3:
        coincidencias.append({
            "archivo": archivo,
            "termino_diccionario": terminos[j],
            "similitud": round(score, 3),
            "tipo": "top3"
        })

# 9. Crear DataFrame y eliminar duplicados
df_resultado = pd.DataFrame(coincidencias).drop_duplicates()

# 10. Guardar en CSV
df_resultado.to_csv("coincidencias_caso.csv", index=False, encoding="utf-8")
print("✅ CSV generado: coincidencias_caso.csv")



✅ CSV generado: coincidencias_caso.csv


In [ ]:
8. Librería Python

In [1]:
!pip install -q sentence-transformers scikit-learn pandas openpyxl


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 48.1 MB/s eta 0:00:00


In [15]:
# === 1. Crear carpeta de la librería ===
!mkdir traductor_sdh

# === 2. Crear traductor.py ===
with open('traductor_sdh/traductor.py', 'w', encoding='utf-8') as f:
    f.write('''\
import os
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

class TraductorSDH:
    def __init__(self, ruta_excel):
        self.df = pd.read_excel(ruta_excel)
        self.modelo = SentenceTransformer('all-MiniLM-L6-v2')

        self.terminos_ingles = self.df['English'].astype(str).tolist()
        self.terminos_espanol = self.df['Spanish'].astype(str).tolist()
        self.embeddings_diccionario = self.modelo.encode(self.terminos_ingles, convert_to_tensor=True)

    def cargar_corpus_desde_carpeta(self, carpeta):
        textos = []
        nombres_archivos = []
        for archivo in os.listdir(carpeta):
            if archivo.endswith(".txt"):
                ruta = os.path.join(carpeta, archivo)
                with open(ruta, 'r', encoding='utf-8') as f:
                    contenido = f.read()
                    textos.append(contenido)
                    nombres_archivos.append(archivo)
        return textos, nombres_archivos

    def analizar_corpus(self, lista_textos, nombres_archivos, umbral=0.5):
        resultados = []
        embeddings_corpus = self.modelo.encode(lista_textos, convert_to_tensor=True)
        similitudes = cosine_similarity(embeddings_corpus, self.embeddings_diccionario)

        for i, fila in enumerate(similitudes):
            for j, score in enumerate(fila):
                if score >= umbral:
                    resultados.append({
                        'archivo': nombres_archivos[i],
                        'frase_corpus': lista_textos[i],
                        'termino_diccionario': self.terminos_ingles[j],
                        'traduccion_espanol': self.terminos_espanol[j],
                        'similitud': round(float(score), 4)
                    })
        return resultados

    def exportar_csv(self, resultados, salida_csv='resultados_sdh.csv'):
        df_resultados = pd.DataFrame(resultados)
        df_resultados.to_csv(salida_csv, index=False, encoding='utf-8')
''')

# === 3. Crear __init__.py ===
with open('traductor_sdh/__init__.py', 'w') as f:
    f.write('from traductor import TraductorSDH')

# === 4. Crear setup.py ===
with open('setup.py', 'w', encoding='utf-8') as f:
    f.write('''\
from setuptools import setup, find_packages

setup(
    name='traductor_sdh',
    version='0.1',
    packages=find_packages(),
    install_requires=[
        'pandas',
        'scikit-learn',
        'sentence-transformers',
        'openpyxl'
    ],
    author='Tu Nombre',
    author_email='tuemail@example.com',
    description='Librería para traducir y detectar determinantes sociales de la salud (SDH)',
    long_description='Traduce y encuentra similitudes entre textos y un diccionario SDH personalizado.',
    long_description_content_type='text/markdown',
    url='https://github.com/patbedmar/traductor_sdh',
    classifiers=[
        'Programming Language :: Python :: 3',
        'Operating System :: OS Independent',
    ],
)
''')



In [16]:
# Instalar herramienta para crear .whl
!pip install -q build

# Generar el archivo .whl
!python setup.py bdist_wheel


running bdist_wheel
running build
running build_py
creating build/lib/traductor_sdh
copying traductor_sdh/traductor.py -> build/lib/traductor_sdh
copying traductor_sdh/__init__.py -> build/lib/traductor_sdh
/usr/local/lib/python3.11/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
installing to build/bdist.linux-x86_64/wheel
running install
running install_lib
creating build/bdist.linux-x86_64/wheel
creating build/bdist.linux-x86_64/wheel/traductor_sdh
copying build/lib/traductor_sdh/tradu

In [17]:
# Instalar el paquete desde la carpeta dist/
!pip install dist/traductor_sdh-0.1-py3-none-any.whl


Processing ./dist/traductor_sdh-0.1-py3-none-any.whl


In [19]:
with open('traductor_sdh/__init__.py', 'w') as f:
    f.write('from traductor_sdh.traductor import TraductorSDH')


9. Usar librería con corpus 20 artículos

In [24]:
from traductor_sdh import TraductorSDH

# Cargar el diccionario de términos
traductor = TraductorSDH('SDOH_Zcodes_traducciones.xlsx')

# Leer los textos del corpus
textos, archivos = traductor.cargar_corpus_desde_carpeta('txt')

# Analizar con similitud >= 0.5
resultados = traductor.analizar_corpus(textos, archivos, umbral=0.5)
resultados2= traductor.analizar_corpus(textos, archivos, umbral=0.1)

# Guardar en CSV
traductor.exportar_csv(resultados, salida_csv='resultados_sdh.csv')
traductor.exportar_csv(resultados2, salida_csv='resultados2_sdh.csv')


10. Usar librería caso práctico

In [25]:
from traductor_sdh import TraductorSDH

# Cargar el diccionario de términos
traductor = TraductorSDH('SDOH_Zcodes_traducciones.xlsx')

# Leer los textos del corpus
textos, archivos = traductor.cargar_corpus_desde_carpeta('caso')

# Analizar con similitud >= 0.5
resultados = traductor.analizar_corpus(textos, archivos, umbral=0.5)
resultados2= traductor.analizar_corpus(textos, archivos, umbral=0.1)

# Guardar en CSV
traductor.exportar_csv(resultados, salida_csv='resultadoscaso_sdh.csv')
traductor.exportar_csv(resultados2, salida_csv='resultados2caso_sdh.csv')


In [26]:
from google.colab import files
files.download('dist/traductor_sdh-0.1-py3-none-any.whl')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>